# Part 4: Monte Carlo Methods

In this notebook, we'll learn **Monte Carlo (MC) methods** - our first **model-free** algorithms that learn from experience without knowing the environment dynamics.

## Recap from Notebook 03
- **Dynamic Programming** algorithms (Policy Evaluation, Policy Iteration, Value Iteration) find optimal policies by computation
- **Model-based learning** requires complete knowledge of MDP dynamics (P and R)
- **Bellman equations** allow us to compute exact value functions through iterative updates
- **Convergence guarantees**: DP methods are guaranteed to find π* given the full model
- **Limitation**: In most real-world problems, we don't know P and R perfectly

## What This Notebook Covers
- Why model-free methods are important
- Monte Carlo prediction (policy evaluation)
- First-visit vs Every-visit MC
- Monte Carlo control (finding optimal policy)
- Exploring starts and ε-soft policies

## What This Notebook Does NOT Cover

| Topic | Why Not Here | How It Differs From What We Cover |
|-------|--------------|-----------------------------------|
| **Bootstrapping methods (TD learning)** | Monte Carlo waits until episode ends to update values. Temporal Difference methods update after each step, which is more efficient. This added complexity comes in the next notebook. | We compute returns G_t by summing rewards to episode end. TD methods estimate returns using V(s') instead of waiting — they "bootstrap" from current value estimates. This enables online learning but introduces bias. |
| **Online learning and continuing tasks** | MC requires complete episodes, making it suitable only for episodic tasks. Continuing tasks (no terminal state) need different approaches. | We wait for episode termination to compute total return. Continuing tasks like stock trading or robot control run indefinitely — they need methods like TD learning that update during the episode. |
| **Deep reinforcement learning** | We use tables to store exact Q(s,a) values. Deep RL uses neural networks to approximate value functions for high-dimensional spaces. | In this notebook, we maintain Q-tables for all state-action pairs (feasible for FrozenLake). Deep RL uses networks f(s,a;θ) for millions of states — essential for Atari, robotics, but builds on MC foundations. |
| **Importance sampling variants** | We use on-policy learning where behavior policy equals target policy. Off-policy learning with importance sampling is more complex but more data-efficient. | Our ε-greedy policy both generates experience AND improves. Importance sampling allows learning π* while following exploratory policy μ — more sample-efficient but requires weighted averaging. |

## Preview: Learning from Experience

Dynamic Programming was powerful but required knowing the MDP model. Now we'll learn our **first model-free method**:

**Monte Carlo (MC) methods:**
- Learn value functions from **sampled episodes** (actual experience)
- No need to know transition probabilities P or reward function R
- Trade exact computation for sample-based learning
- Use **averaging** to estimate expected returns

The key idea: If we run many episodes and average the returns, we'll converge to the true value function!

This is the beginning of **practical RL** — learning from interaction without needing a model.

## How to Read This Notebook
1. **Model-free intuition**: Understand how we learn from experience instead of computation
2. **MC algorithms**: See how first-visit MC prediction and control work step-by-step
3. **Implementation details**: Run code to watch Q-values and policies improve over episodes
4. **Convergence behavior**: Observe how sample averaging leads to correct values

Let's begin!

## Prerequisites
- Understanding of MDPs and value functions (Notebooks 01-02)
- Dynamic Programming concepts (Notebook 03)

## Setup

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(42)

print("Setup complete!")

In [ ]:
# Create environment and helper variables
env = gym.make("FrozenLake-v1", is_slippery=True)

n_states = env.observation_space.n
n_actions = env.action_space.n
action_names = ['LEFT', 'DOWN', 'RIGHT', 'UP']
action_arrows = ['←', '↓', '→', '↑']

print("FrozenLake Environment")
print("=" * 40)
print(f"States: {n_states}")
print(f"Actions: {n_actions}")

In [ ]:
# Visualization helper functions
def plot_value_function(V, title="Value Function", ax=None):
    """Plot value function as a heatmap."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    V_grid = V.reshape(nrow, ncol)
    
    im = ax.imshow(V_grid, cmap='RdYlGn', vmin=0, vmax=max(V.max(), 0.01))
    plt.colorbar(im, ax=ax, shrink=0.8)
    
    for i in range(nrow):
        for j in range(ncol):
            state = i * ncol + j
            cell = desc[i, j]
            color = 'white' if V_grid[i, j] < V.max() / 2 else 'black'
            ax.text(j, i, f'{cell}\n{V[state]:.3f}', ha='center', va='center',
                   fontsize=9, color=color)
    
    ax.set_xticks(range(ncol))
    ax.set_yticks(range(nrow))
    ax.set_title(title)
    return ax

def plot_policy(Q, title="Policy", ax=None):
    """Plot policy derived from Q-values."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    for i in range(nrow):
        for j in range(ncol):
            state = i * ncol + j
            cell = desc[i, j]
            
            rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                 facecolor=colors.get(cell, 'white'), edgecolor='black')
            ax.add_patch(rect)
            
            best_action = np.argmax(Q[state])
            
            if cell not in ['H', 'G']:
                ax.text(j + 0.5, nrow - 1 - i + 0.5, 
                       f'{cell}\n{action_arrows[best_action]}',
                       ha='center', va='center', fontsize=14, fontweight='bold')
            else:
                ax.text(j + 0.5, nrow - 1 - i + 0.5, cell,
                       ha='center', va='center', fontsize=14, fontweight='bold')
    
    ax.set_xlim(0, ncol)
    ax.set_ylim(0, nrow)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title)
    return ax

print("Visualization functions ready!")

---
# 1. Why Model-Free Methods?

## Limitations of Dynamic Programming

Dynamic Programming (Policy Iteration, Value Iteration) requires:
- Complete knowledge of transition probabilities $P_{ss'}^a$
- Complete knowledge of reward function $R_s^a$

In many real-world problems:
- The environment model is **unknown**
- The model is too **complex** to use directly
- Building the model is too **expensive**

## Model-Free Learning

**Model-free** methods learn directly from experience:
- No need to know P or R
- Learn from sampled episodes
- Can handle unknown environments

Monte Carlo methods are one class of model-free algorithms.

---
# 2. Monte Carlo Prediction

**Problem**: Estimate $V^\pi(s)$ or $Q^\pi(s,a)$ for a given policy $\pi$, without knowing the model.

## Key Idea

The value function is the **expected return**:

$$V^\pi(s) = E_\pi[G_t | S_t = s]$$

Monte Carlo estimates this expectation by **averaging sample returns**:

$$V^\pi(s) \approx \frac{1}{N} \sum_{i=1}^{N} G_t^{(i)}$$

where $G_t^{(i)}$ is the return from state $s$ in the $i$-th episode.

## Requirements

- Episodes must **terminate** (finite episodes)
- We learn from **complete episodes** (unlike TD methods)
- Only updates happen at the **end of episodes**

## First-Visit vs Every-Visit MC

When a state $s$ is visited multiple times in an episode:

**First-Visit MC**: Only use the return from the **first** time $s$ is visited

**Every-Visit MC**: Use returns from **every** time $s$ is visited

Both converge to $V^\pi(s)$ as the number of episodes → ∞

In [ ]:
def generate_episode(env, policy):
    """
    Generate an episode following the given policy.
    
    Args:
        env: Gymnasium environment
        policy: Policy array π[s,a] with probabilities or π[s] with action
    
    Returns:
        episode: List of (state, action, reward) tuples
    """
    episode = []
    state, _ = env.reset()
    done = False
    
    while not done:
        # Get action from policy
        if len(policy.shape) == 1:
            action = int(policy[state])
        else:
            action = np.random.choice(len(policy[state]), p=policy[state])
        
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        done = terminated or truncated
    
    return episode

In [ ]:
# Demonstrate episode generation
uniform_policy = np.ones((n_states, n_actions)) / n_actions

print("Example Episodes with Random Policy")
print("=" * 60)

for i in range(5):
    episode = generate_episode(env, uniform_policy)
    states = [s for s, a, r in episode]
    actions = [action_arrows[a] for s, a, r in episode]
    rewards = [r for s, a, r in episode]
    total_reward = sum(rewards)
    
    print(f"\nEpisode {i+1} (length={len(episode)}, reward={total_reward}):")
    print(f"  States:  {states}")
    print(f"  Actions: {actions}")
    print(f"  Rewards: {rewards}")

In [ ]:
def mc_prediction_first_visit(env, policy, gamma, n_episodes, verbose=False):
    """
    First-Visit Monte Carlo Prediction for estimating V^π.
    
    Args:
        env: Gymnasium environment
        policy: Policy to evaluate
        gamma: Discount factor
        n_episodes: Number of episodes to sample
        verbose: Print progress
    
    Returns:
        V: Estimated state value function
        V_history: Value function at intervals for visualization
    """
    n_states = env.observation_space.n
    
    # Store returns for each state
    returns_sum = np.zeros(n_states)
    returns_count = np.zeros(n_states)
    V = np.zeros(n_states)
    
    V_history = [V.copy()]
    
    for episode_num in range(n_episodes):
        # Generate an episode
        episode = generate_episode(env, policy)
        
        # Calculate returns for each state visited
        states_visited = set()
        G = 0
        
        # Go backwards through the episode
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # First-visit: only count if this is the first occurrence of state in episode
            if state not in states_visited:
                states_visited.add(state)
                returns_sum[state] += G
                returns_count[state] += 1
        
        # Update V
        for s in range(n_states):
            if returns_count[s] > 0:
                V[s] = returns_sum[s] / returns_count[s]
        
        # Save history at intervals
        if (episode_num + 1) % (n_episodes // 10) == 0:
            V_history.append(V.copy())
            if verbose:
                print(f"Episode {episode_num + 1}/{n_episodes}")
    
    return V, V_history

In [ ]:
# Run MC prediction for random policy
print("Monte Carlo Prediction (First-Visit)")
print("=" * 50)
print("Evaluating uniform random policy...\n")

start_time = time.time()
V_mc, V_history = mc_prediction_first_visit(env, uniform_policy, gamma=0.99, 
                                             n_episodes=50000, verbose=True)
mc_time = time.time() - start_time

print(f"\nTime taken: {mc_time:.2f} seconds")
print(f"\nEstimated V^π (random policy):")
print(V_mc.reshape(4, 4).round(4))

In [ ]:
# Compare with DP solution (if we had the model)
# First, get the "true" values using DP
def extract_mdp(env):
    n_s = env.observation_space.n
    n_a = env.action_space.n
    P = np.zeros((n_s, n_a, n_s))
    R = np.zeros((n_s, n_a))
    for s in range(n_s):
        for a in range(n_a):
            for prob, next_s, reward, done in env.unwrapped.P[s][a]:
                P[s, a, next_s] += prob
                R[s, a] += prob * reward
    return P, R

def policy_evaluation_dp(P, R, policy, gamma, theta=1e-8):
    n_states = P.shape[0]
    n_actions = P.shape[1]
    V = np.zeros(n_states)
    
    while True:
        V_new = np.zeros(n_states)
        for s in range(n_states):
            for a in range(n_actions):
                V_new[s] += policy[s, a] * (R[s, a] + gamma * np.sum(P[s, a] * V))
        
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
    return V

P, R = extract_mdp(env)
V_true = policy_evaluation_dp(P, R, uniform_policy, gamma=0.99)

print("Comparison: MC Prediction vs DP (True Values)")
print("=" * 60)
print(f"{'State':<8} {'MC Estimate':>15} {'True (DP)':>15} {'Error':>15}")
print("-" * 60)
for s in range(n_states):
    error = abs(V_mc[s] - V_true[s])
    print(f"{s:<8} {V_mc[s]:>15.4f} {V_true[s]:>15.4f} {error:>15.4f}")

print(f"\nMean Absolute Error: {np.mean(np.abs(V_mc - V_true)):.4f}")

In [ ]:
# Visualize MC convergence
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Plot value function evolution
episodes_at = [0, 5000, 10000, 25000, 50000]
for idx, (ax, ep) in enumerate(zip(axes.flat[:-1], episodes_at)):
    hist_idx = min(idx, len(V_history)-1)
    plot_value_function(V_history[hist_idx], title=f"After {ep} episodes", ax=ax)

# Plot comparison with true values
ax = axes.flat[-1]
x = np.arange(n_states)
width = 0.35
ax.bar(x - width/2, V_mc, width, label='MC Estimate', color='steelblue')
ax.bar(x + width/2, V_true, width, label='True (DP)', color='orange')
ax.set_xlabel('State')
ax.set_ylabel('Value')
ax.set_title('MC vs True Values')
ax.legend()

plt.suptitle("Monte Carlo Prediction Convergence (50,000 episodes)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
# 3. Monte Carlo Estimation of Q-values

For **control** (finding the optimal policy), we need to estimate **action-values** $Q^\pi(s,a)$, not just state-values.

Why? Because to improve a policy, we need to know how good each action is:

$$\pi'(s) = \arg\max_a Q^\pi(s, a)$$

With just $V(s)$, we would need the model to compute:
$$Q(s,a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a V(s')$$

But we're model-free! So we estimate Q directly.

In [ ]:
def mc_prediction_Q(env, policy, gamma, n_episodes):
    """
    First-Visit Monte Carlo Prediction for Q-values.
    
    Returns:
        Q: Estimated action-value function Q[s, a]
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    returns_sum = np.zeros((n_states, n_actions))
    returns_count = np.zeros((n_states, n_actions))
    Q = np.zeros((n_states, n_actions))
    
    for _ in range(n_episodes):
        episode = generate_episode(env, policy)
        
        # Track visited (state, action) pairs
        sa_visited = set()
        G = 0
        
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # First-visit check
            if (state, action) not in sa_visited:
                sa_visited.add((state, action))
                returns_sum[state, action] += G
                returns_count[state, action] += 1
        
        # Update Q
        for s in range(n_states):
            for a in range(n_actions):
                if returns_count[s, a] > 0:
                    Q[s, a] = returns_sum[s, a] / returns_count[s, a]
    
    return Q

# Estimate Q for random policy
print("Estimating Q^π for random policy...")
Q_random = mc_prediction_Q(env, uniform_policy, gamma=0.99, n_episodes=50000)

print("\nQ-values for state 0 (start):")
for a in range(n_actions):
    print(f"  Q(0, {action_names[a]}) = {Q_random[0, a]:.4f}")

---
# 4. Monte Carlo Control

**Goal**: Find the optimal policy $\pi^*$ using only experience.

## Approach: Generalized Policy Iteration

Like Policy Iteration, we alternate between:
1. **Policy Evaluation**: Estimate $Q^\pi$ using MC
2. **Policy Improvement**: Make policy greedy with respect to Q

## The Exploration Problem

If our policy is deterministic, we might never visit some (state, action) pairs!

**Solution 1: Exploring Starts**
- Start each episode from a random (state, action) pair
- Ensures all pairs have chance to be visited

**Solution 2: ε-soft Policies**
- Always have non-zero probability of selecting any action
- E.g., ε-greedy: with prob ε choose random, else choose best

In [ ]:
def mc_control_epsilon_greedy(env, gamma, n_episodes, epsilon=0.1, 
                               epsilon_decay=0.9999, min_epsilon=0.01):
    """
    Monte Carlo Control with ε-greedy exploration.
    
    Args:
        env: Gymnasium environment
        gamma: Discount factor
        n_episodes: Number of episodes
        epsilon: Initial exploration rate
        epsilon_decay: Decay rate for epsilon
        min_epsilon: Minimum epsilon value
    
    Returns:
        Q: Learned action-value function
        policy: Learned policy
        stats: Training statistics
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    # Initialize Q arbitrarily and returns tracking
    Q = np.zeros((n_states, n_actions))
    returns_sum = np.zeros((n_states, n_actions))
    returns_count = np.zeros((n_states, n_actions))
    
    # Statistics
    episode_rewards = []
    episode_lengths = []
    epsilons = []
    
    for episode_num in range(n_episodes):
        # Generate episode using ε-greedy policy derived from Q
        episode = []
        state, _ = env.reset()
        done = False
        
        while not done:
            # ε-greedy action selection
            if np.random.random() < epsilon:
                action = np.random.randint(n_actions)
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state
            done = terminated or truncated
        
        # Record stats
        episode_rewards.append(sum(r for _, _, r in episode))
        episode_lengths.append(len(episode))
        epsilons.append(epsilon)
        
        # Update Q using first-visit MC
        sa_visited = set()
        G = 0
        
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            if (state, action) not in sa_visited:
                sa_visited.add((state, action))
                returns_sum[state, action] += G
                returns_count[state, action] += 1
                Q[state, action] = returns_sum[state, action] / returns_count[state, action]
        
        # Decay epsilon
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    
    # Extract greedy policy
    policy = np.zeros((n_states, n_actions))
    for s in range(n_states):
        policy[s, np.argmax(Q[s])] = 1.0
    
    stats = {
        'episode_rewards': episode_rewards,
        'episode_lengths': episode_lengths,
        'epsilons': epsilons
    }
    
    return Q, policy, stats

In [ ]:
# Run MC Control
print("Monte Carlo Control with ε-greedy")
print("=" * 50)

start_time = time.time()
Q_mc, policy_mc, stats = mc_control_epsilon_greedy(
    env, gamma=0.99, n_episodes=100000, 
    epsilon=1.0, epsilon_decay=0.99995, min_epsilon=0.01
)
mc_control_time = time.time() - start_time

print(f"Training time: {mc_control_time:.2f} seconds")
print(f"Final epsilon: {stats['epsilons'][-1]:.4f}")

In [ ]:
# Plot training progress
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Smooth rewards with moving average
window = 1000
rewards_smooth = np.convolve(stats['episode_rewards'], 
                              np.ones(window)/window, mode='valid')

# Episode rewards
axes[0, 0].plot(rewards_smooth)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Reward (moving avg)')
axes[0, 0].set_title(f'Learning Curve (window={window})')

# Epsilon decay
axes[0, 1].plot(stats['epsilons'])
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Epsilon')
axes[0, 1].set_title('Exploration Rate Decay')

# Final Q-values heatmap
V_from_Q = np.max(Q_mc, axis=1)
plot_value_function(V_from_Q, title="Learned V* = max_a Q(s,a)", ax=axes[1, 0])

# Learned policy
plot_policy(Q_mc, title="Learned Policy", ax=axes[1, 1])

plt.suptitle("Monte Carlo Control Results (100,000 episodes)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the learned policy
def evaluate_policy(env, Q, n_episodes=10000):
    """Evaluate a greedy policy derived from Q."""
    rewards = []
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = np.argmax(Q[state])
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated
        
        rewards.append(total_reward)
    
    return np.array(rewards)

# Compare learned policy with optimal (from DP) and random
print("Policy Evaluation Comparison")
print("=" * 50)

# MC learned policy
rewards_mc = evaluate_policy(env, Q_mc, n_episodes=10000)
print(f"MC Policy: Success rate = {np.mean(rewards_mc)*100:.2f}%")

# Random policy
Q_random = np.random.rand(n_states, n_actions)
rewards_random = evaluate_policy(env, Q_random, n_episodes=10000)
print(f"Random Policy: Success rate = {np.mean(rewards_random)*100:.2f}%")

# Optimal policy (from Value Iteration)
def value_iteration(P, R, gamma, theta=1e-8):
    n_states, n_actions = R.shape
    V = np.zeros(n_states)
    while True:
        V_new = np.zeros(n_states)
        for s in range(n_states):
            V_new[s] = np.max([R[s, a] + gamma * np.sum(P[s, a] * V) for a in range(n_actions)])
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
    # Extract Q
    Q = np.zeros((n_states, n_actions))
    for s in range(n_states):
        for a in range(n_actions):
            Q[s, a] = R[s, a] + gamma * np.sum(P[s, a] * V)
    return Q

Q_optimal = value_iteration(P, R, gamma=0.99)
rewards_optimal = evaluate_policy(env, Q_optimal, n_episodes=10000)
print(f"Optimal Policy (DP): Success rate = {np.mean(rewards_optimal)*100:.2f}%")

In [ ]:
# Visualize policy comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

policies = ['Random', 'MC Learned', 'Optimal (DP)']
success_rates = [np.mean(rewards_random)*100, np.mean(rewards_mc)*100, np.mean(rewards_optimal)*100]
colors = ['gray', 'steelblue', 'green']

# Bar chart
bars = axes[0].bar(policies, success_rates, color=colors, edgecolor='black')
axes[0].set_ylabel('Success Rate (%)')
axes[0].set_title('Policy Performance Comparison')
axes[0].set_ylim(0, 100)
for bar, rate in zip(bars, success_rates):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{rate:.1f}%', ha='center', fontweight='bold')

# MC learned policy
plot_policy(Q_mc, title="MC Learned Policy", ax=axes[1])

# Optimal policy
plot_policy(Q_optimal, title="Optimal Policy (DP)", ax=axes[2])

plt.tight_layout()
plt.show()

---
# 5. Exploring Starts MC Control

An alternative to ε-greedy is **Exploring Starts**: every (state, action) pair has non-zero probability of being the starting point of an episode.

This guarantees all pairs are visited infinitely often, allowing convergence to the true optimal policy.

In [ ]:
def mc_control_exploring_starts(env, gamma, n_episodes):
    """
    Monte Carlo Control with Exploring Starts.
    
    Note: This requires ability to set the initial state,
    which isn't always possible in real environments.
    """
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    Q = np.zeros((n_states, n_actions))
    returns_sum = np.zeros((n_states, n_actions))
    returns_count = np.zeros((n_states, n_actions))
    
    episode_rewards = []
    
    for _ in range(n_episodes):
        # Exploring start: random initial state and action
        # Note: FrozenLake doesn't support setting initial state,
        # so we'll simulate by using the normal start but random first action
        state, _ = env.reset()
        
        # Random first action (exploring start)
        first_action = np.random.randint(n_actions)
        
        # Generate episode
        episode = []
        action = first_action
        done = False
        
        while not done:
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state
            done = terminated or truncated
            
            if not done:
                # Follow greedy policy after first action
                action = np.argmax(Q[state])
        
        episode_rewards.append(sum(r for _, _, r in episode))
        
        # Update Q
        sa_visited = set()
        G = 0
        
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = gamma * G + r
            
            if (s, a) not in sa_visited:
                sa_visited.add((s, a))
                returns_sum[s, a] += G
                returns_count[s, a] += 1
                Q[s, a] = returns_sum[s, a] / returns_count[s, a]
    
    return Q, episode_rewards

# Run Exploring Starts MC
print("Monte Carlo Control with Exploring Starts")
print("=" * 50)

Q_es, rewards_es = mc_control_exploring_starts(env, gamma=0.99, n_episodes=100000)

# Evaluate
rewards_es_eval = evaluate_policy(env, Q_es, n_episodes=10000)
print(f"Exploring Starts Policy: Success rate = {np.mean(rewards_es_eval)*100:.2f}%")

---
# 6. MC vs DP: Key Differences

| Aspect | Dynamic Programming | Monte Carlo |
|--------|--------------------|--------------|
| **Model** | Requires complete model | Model-free |
| **Updates** | Bootstraps (uses estimated values) | Uses actual returns |
| **Episodes** | Can update mid-episode | Must wait for episode end |
| **Variance** | Lower (uses expectations) | Higher (uses samples) |
| **Bias** | Biased by initialization | Unbiased (true returns) |

In [ ]:
# Compare convergence characteristics
print("MC Characteristics Demonstration")
print("=" * 50)

# Run MC with different numbers of episodes
episode_counts = [1000, 5000, 10000, 50000, 100000]
mc_errors = []

for n_ep in episode_counts:
    Q_temp, _, _ = mc_control_epsilon_greedy(env, gamma=0.99, n_episodes=n_ep,
                                             epsilon=1.0, epsilon_decay=0.9999, min_epsilon=0.01)
    # Compare with optimal Q
    error = np.mean(np.abs(Q_temp - Q_optimal))
    mc_errors.append(error)
    print(f"{n_ep:>7} episodes: Mean |Q - Q*| = {error:.4f}")

# Plot
plt.figure(figsize=(10, 5))
plt.plot(episode_counts, mc_errors, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Episodes')
plt.ylabel('Mean Absolute Error in Q')
plt.title('MC Control Convergence to Optimal Q')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

---
# Summary and Concept Map

In this notebook, we learned Monte Carlo methods - our first model-free algorithms that learn from experience!

```
MONTE CARLO METHODS for RL
===========================

Key Innovation: Learn from EXPERIENCE, not computation
No need for MDP model (P, R) - completely MODEL-FREE!
────────────────────────────────────────────────────


CORE IDEA: SAMPLE-BASED ESTIMATION
──────────────────────────────────
Value = Expected Return
V^π(s) = E_π[G_t | S_t = s]

DP: Uses model to compute expectation
    V(s) = Σ π(a|s) [R^a_s + γ Σ P^a_{ss'} V(s')]
    
MC: Samples episodes and averages actual returns
    V^π(s) ≈ (1/N) Σ_{i=1}^N G_t^(i)
    

MC PREDICTION (Policy Evaluation)
─────────────────────────────────
Problem: Estimate V^π(s) or Q^π(s,a) without model

Algorithm: 
1. Generate episode following π: S_0, A_0, R_1, ..., S_T
2. For each state s visited:
   - Calculate return: G_t = R_{t+1} + γR_{t+2} + ... + γ^{T-t-1}R_T
   - Update value estimate: V(s) = average of all G_t from s

Two variants:
- First-visit MC: Count only first visit to s in episode
- Every-visit MC: Count all visits to s in episode

Key property: UNBIASED estimates (uses actual returns)
Limitation: HIGH VARIANCE (different episodes give different G_t)


MC CONTROL (Finding Optimal Policy)
───────────────────────────────────
Problem: Find π* without knowing P or R

Challenge: Need Q(s,a), not just V(s), for improvement
Why? Can't compute max_a [R + γΣP V] without model!

Solution: Estimate Q^π(s,a) directly from episodes

Algorithm (Generalized Policy Iteration):
1. Initialize Q(s,a) arbitrarily, π ε-greedy
2. Loop:
   a. Generate episode with current policy
   b. For each (s,a) pair visited:
      - Calculate return G_t
      - Update Q(s,a) = average of returns from (s,a)
   c. Improve policy: make ε-greedy w.r.t. Q
3. Return learned Q and π


EXPLORATION STRATEGIES
─────────────────────
Problem: Deterministic policy never explores alternatives!

Solution 1: Exploring Starts
- Every (s,a) pair has nonzero probability of starting episode
- Guarantees all pairs visited infinitely often
- Limitation: Not feasible in many real environments

Solution 2: ε-Greedy Policies
- With probability ε: choose random action (explore)
- With probability 1-ε: choose best action (exploit)
- Ensures all actions tried infinitely often
- ε decay: Start high (explore), decay to low (exploit)

Trade-off: Exploration vs Exploitation
- Too much exploration: Suboptimal actions slow learning
- Too little exploration: Miss better actions, get stuck


MC vs DP: KEY DIFFERENCES
─────────────────────────
| Aspect | Dynamic Programming | Monte Carlo |
|--------|-------------------|-------------|
| **Model** | Requires P and R | Model-free |
| **Updates** | Bootstraps (uses V estimates) | Uses actual returns |
| **Timing** | Can update any time | Wait for episode end |
| **Episodes** | Not needed | MUST have episodes |
| **Variance** | Low (expectations) | High (sampling) |
| **Bias** | Biased by initialization | Unbiased (true G_t) |
| **Computation** | Iterate over all states | Sample episodes |
| **Task type** | Any MDP | EPISODIC only |

When to use MC:
✓ Don't know environment model
✓ Have episodic tasks (games, simulations)
✓ Can tolerate high variance
✓ Want unbiased estimates

When to use DP:
✓ Know complete MDP model
✓ Need low variance
✓ Have continuing tasks
✓ Want computational efficiency


CONVERGENCE PROPERTIES
──────────────────────
First-visit MC converges to V^π(s) as episodes → ∞
- Law of Large Numbers: sample mean → true mean
- Guaranteed for episodic tasks with bounded returns

MC Control converges to Q* if:
1. All (s,a) pairs visited infinitely often (exploration)
2. Policy improves greedily w.r.t. Q
3. Episodes are finite and have bounded returns

Practical convergence:
- Needs MANY episodes (10,000s to 100,000s)
- ε-greedy with decay balances exploration/exploitation
- Variance decreases as 1/√N (slow!)


IMPLEMENTATION INSIGHTS
───────────────────────
Data structure: Track returns_sum[s,a] and returns_count[s,a]
Update: Q[s,a] = returns_sum[s,a] / returns_count[s,a]

First-visit check: Use set() to track visited (s,a) pairs
Backward iteration: Process episode in reverse for efficiency
  for t in reversed(range(len(episode))):
      G = γ*G + reward[t]

ε-decay schedule: 
- Start: ε = 1.0 (full exploration)
- Decay: ε = max(min_ε, ε * decay_rate)
- End: ε ≈ 0.01 (mostly exploitation)

Episode generation:
- Follow current policy (ε-greedy)
- Store (state, action, reward) tuples
- Continue until terminal state reached
```

## Key Takeaways

1. **Model-free learning**: MC learns from sampled experience, no P or R needed
2. **Episode-based**: Must wait for complete episodes before updating values
3. **Unbiased but high variance**: Uses actual returns (unbiased) but varies across episodes
4. **Exploration is crucial**: ε-greedy or exploring starts ensure all actions are tried
5. **Q-learning not V-learning**: Need Q(s,a) for model-free control
6. **Episodic tasks only**: MC requires episodes to terminate (have finite length)

## Limitations of Monte Carlo

- **Episodic tasks only**: Cannot handle continuing tasks (infinite horizon)
- **Episode completion required**: Must wait until end to update (slow for long episodes)
- **High variance**: Sample returns can vary significantly between episodes
- **Sample inefficiency**: Each return updates only one (s,a) pair per episode
- **Slow convergence**: Needs many episodes to average out variance

## What's Next?

Monte Carlo was a huge step forward (model-free!), but has limitations:
- Only works for episodic tasks
- Must wait for episode to end
- High variance slows learning

In the next notebook (**05_temporal_difference.ipynb**), we'll learn **Temporal Difference (TD) methods**:
- Update **every step** (not just at episode end)
- Work for **continuing** tasks (non-episodic)
- **Lower variance** than MC (bootstrap from estimates)
- Include famous algorithms: **SARSA** and **Q-Learning**

TD methods combine the best of both worlds:
- Model-free like MC
- Bootstrap like DP
- Online learning capability

In [ ]:
---
# Your Turn

Now it's time to test your understanding with some hands-on exercises!

## Exercise 1: Implement First-Visit MC Prediction for Custom Policy Evaluation

In the notebook, we evaluated a uniform random policy. Now implement first-visit MC prediction to evaluate a different policy.

**Task**: Complete the code below to evaluate a "cautious" policy that prefers RIGHT and DOWN over LEFT and UP.

```python
# YOUR CODE HERE
# Define a cautious policy: P(RIGHT) = 0.4, P(DOWN) = 0.4, P(LEFT) = 0.1, P(UP) = 0.1

def create_cautious_policy(n_states, n_actions):
    """
    Create a cautious policy that prefers RIGHT and DOWN.
    
    Returns: policy array of shape (n_states, n_actions)
    """
    policy = np.zeros((n_states, n_actions))
    
    # TODO: Fill in the policy probabilities
    # Hint: For all states, set probabilities as [0.1, 0.4, 0.4, 0.1] for [LEFT, DOWN, RIGHT, UP]
    
    pass  # Replace with your implementation

# TODO: Run MC prediction with this cautious policy for 50,000 episodes
# Hint: Use mc_prediction_first_visit function from the notebook

# TODO: Compare V(start_state) for cautious vs random policy
# Which one has higher value at the start state?
```

<details>
<summary>Click to see hint</summary>

The cautious policy should:
- Have same probabilities for all states: [0.1, 0.4, 0.4, 0.1]
- Use the `mc_prediction_first_visit` function with gamma=0.99
- Compare the value at state 0 (start state)

Remember that RIGHT generally moves toward the goal in FrozenLake, so this policy should perform better than random!

</details>

<details>
<summary>Click to see solution</summary>

```python
def create_cautious_policy(n_states, n_actions):
    """Create cautious policy that prefers RIGHT and DOWN."""
    policy = np.zeros((n_states, n_actions))
    # For all states: [LEFT, DOWN, RIGHT, UP] = [0.1, 0.4, 0.4, 0.1]
    for s in range(n_states):
        policy[s] = [0.1, 0.4, 0.4, 0.1]
    return policy

# Create and evaluate cautious policy
cautious_policy = create_cautious_policy(n_states, n_actions)
V_cautious, _ = mc_prediction_first_visit(env, cautious_policy, gamma=0.99, 
                                          n_episodes=50000, verbose=True)

print("\nComparison:")
print(f"Cautious policy V(0): {V_cautious[0]:.4f}")
print(f"Random policy V(0):   {V_random[0]:.4f}")
print(f"Improvement: {V_cautious[0] - V_random[0]:.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_value_function(V_cautious, title="Cautious Policy V^π", ax=axes[0])
plot_value_function(V_random, title="Random Policy V^π", ax=axes[1])
plt.tight_layout()
plt.show()

# The cautious policy should have higher value at the start!
# Why? Because it moves toward the goal (right and down) more often.
```

**Key Insight**: Even without knowing the optimal policy, choosing actions that seem reasonable (moving toward goal) improves performance. This is why policy design matters in RL!

</details>

## Exercise 2: Experiment with ε-Decay Schedules

The MC control algorithm uses ε-greedy exploration with decay. The decay schedule significantly affects learning performance.

**Task**: Experiment with different ε-decay schedules and compare their convergence.

```python
# YOUR CODE HERE
# Test three decay schedules:
# 1. Fast decay: epsilon_decay = 0.999
# 2. Medium decay: epsilon_decay = 0.9999 (what we used)
# 3. Slow decay: epsilon_decay = 0.99999

decay_rates = [0.999, 0.9999, 0.99999]
results_comparison = []

for decay in decay_rates:
    # TODO: Run mc_control_epsilon_greedy with different decay rates
    # Use n_episodes = 50000 for fair comparison
    # Record: final success rate, final epsilon, learning curve
    pass

# TODO: Plot comparison of learning curves
# TODO: Which decay schedule learns fastest? Which achieves best final performance?
```

<details>
<summary>Click to see hint</summary>

For each decay rate:
1. Run `mc_control_epsilon_greedy` with the decay rate
2. Evaluate the learned policy using `evaluate_policy`
3. Plot the smoothed rewards over episodes
4. Compare final epsilon values

Think about the trade-off:
- Fast decay: Quick convergence but might miss optimal actions
- Slow decay: Better exploration but slower convergence

</details>

<details>
<summary>Click to see solution</summary>

```python
decay_rates = [0.999, 0.9999, 0.99999]
decay_names = ['Fast (0.999)', 'Medium (0.9999)', 'Slow (0.99999)']
results_comparison = []

print("Testing Different ε-Decay Schedules")
print("=" * 70)

for decay, name in zip(decay_rates, decay_names):
    print(f"\nTesting {name}...")
    Q, policy, stats = mc_control_epsilon_greedy(
        env, gamma=0.99, n_episodes=50000,
        epsilon=1.0, epsilon_decay=decay, min_epsilon=0.01
    )
    
    # Evaluate final policy
    rewards_eval = evaluate_policy(env, Q, n_episodes=10000)
    success_rate = np.mean(rewards_eval) * 100
    final_epsilon = stats['epsilons'][-1]
    
    results_comparison.append({
        'name': name,
        'decay': decay,
        'Q': Q,
        'stats': stats,
        'success_rate': success_rate,
        'final_epsilon': final_epsilon
    })
    
    print(f"  Final success rate: {success_rate:.2f}%")
    print(f"  Final epsilon: {final_epsilon:.4f}")

# Plot learning curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Learning curves
window = 1000
for r in results_comparison:
    rewards_smooth = np.convolve(r['stats']['episode_rewards'], 
                                  np.ones(window)/window, mode='valid')
    axes[0].plot(rewards_smooth, label=r['name'], linewidth=2)

axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Reward (moving avg)')
axes[0].set_title('Learning Curves Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Epsilon decay curves
for r in results_comparison:
    axes[1].plot(r['stats']['epsilons'], label=r['name'], linewidth=2)

axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Epsilon')
axes[1].set_title('Exploration Rate Decay')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Final performance comparison
names = [r['name'] for r in results_comparison]
success = [r['success_rate'] for r in results_comparison]
colors = ['red', 'green', 'blue']

bars = axes[2].bar(range(len(names)), success, color=colors, edgecolor='black')
axes[2].set_xticks(range(len(names)))
axes[2].set_xticklabels(names, rotation=15, ha='right')
axes[2].set_ylabel('Success Rate (%)')
axes[2].set_title('Final Performance')
axes[2].set_ylim(0, 100)

for bar, rate in zip(bars, success):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{rate:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("ANALYSIS:")
print("- Fast decay (0.999): Converges quickly but may get stuck in local optimum")
print("- Medium decay (0.9999): Good balance of exploration and exploitation")
print("- Slow decay (0.99999): More exploration, slower convergence but better final policy")
print("\nKey insight: ε-decay schedule is crucial for balancing exploration vs exploitation!")
```

**Key Takeaways**:
- **Fast decay**: Learns quickly early on but stops exploring too soon
- **Slow decay**: Explores more thoroughly, often finds better final policy
- **Trade-off**: Convergence speed vs final performance quality
- In practice: Start with medium decay, tune based on task complexity

</details>

## Exercise 3: Conceptual Question - MC Variance vs DP Bias

**Question**: We learned that Monte Carlo methods have high variance but are unbiased, while Dynamic Programming has low variance but can be biased by initialization. Explain:

1. Why does MC have high variance? Give a concrete example from FrozenLake.
2. What does "unbiased" mean in the context of MC estimation?
3. In what scenarios would you prefer MC's high-variance, unbiased estimates over DP's low-variance, potentially biased estimates?

<details>
<summary>Click to see hint</summary>

Think about:
1. **Variance**: Why do different episodes from the same state give different returns?
2. **Bias**: What does it mean that MC converges to the "true" value?
3. **Trade-offs**: When is it better to have noisy but correct estimates vs smooth but potentially wrong estimates?

Consider:
- Episode 1: Start → Hole (return = 0)
- Episode 2: Start → Goal (return = 1.0)
- Both from same state with same policy!

</details>

<details>
<summary>Click to see detailed answer</summary>

### 1. Why MC has high variance

**High variance** means the estimated value can vary significantly between episodes, even from the same state.

**FrozenLake example**:
```
State 0 (start) with random policy:

Episode 1: S0 → S4 → H (Hole at step 2)
  Return: G = 0
  
Episode 2: S0 → S1 → S2 → S6 → S7 → G (Goal at step 5)
  Return: G = γ^4 * 1.0 = 0.99^4 ≈ 0.96
  
Episode 3: S0 → S0 → S4 → S5 → H (Hole at step 4)
  Return: G = 0

V(S0) after 3 episodes: (0 + 0.96 + 0) / 3 = 0.32
```

The same state gives returns ranging from 0 to 0.96! This is **high variance**.

**Why this happens**:
- Environment stochasticity (slippery ice)
- Policy stochasticity (random actions)
- Different episode trajectories from same state
- Variance decreases only as 1/√N (slow!)

### 2. What "unbiased" means for MC

**Unbiased** means the *expected value* of the estimate equals the true value:
```
E[V_MC(s)] = V^π(s)
```

Even though individual returns vary (high variance), their **average converges to the true value**.

**Why MC is unbiased**:
- Uses actual returns G_t (not estimates)
- Law of Large Numbers: sample mean → population mean
- No bootstrapping (doesn't depend on other estimates)

**DP can be biased**:
```
V_{k+1}(s) = Σ π(a|s) [R + γ Σ P * V_k(s')]
                                    ^^^^
                              Uses estimated values!
```
If V_k is wrong (from initialization), it propagates. Only asymptotically unbiased.

### 3. When to prefer MC over DP

**Prefer MC when**:

✓ **No model available**
- Most important reason! DP requires knowing P and R
- Example: Learning to play a video game (don't know transition dynamics)

✓ **Subset of states matters**
- MC only updates visited states (efficient for large state spaces)
- DP must update all states each iteration
- Example: Chess - only care about good positions, not all 10^47 states

✓ **Want unbiased estimates**
- Critical for statistical guarantees
- Example: A/B testing policies - need true value differences

✓ **Can simulate episodes cheaply**
- If simulation is fast, variance is manageable
- Example: Game emulators, physics simulators

**Prefer DP when**:

✓ **Have complete model** (P, R known)
- Can leverage model for efficiency
- Example: Gridworld puzzles, tic-tac-toe

✓ **Need low variance**
- Noisy estimates are problematic
- Example: Safety-critical applications (robotics)

✓ **Continuing tasks**
- MC requires episodes to end
- Example: Stock trading (market never "terminates")

✓ **Computational efficiency**
- DP can converge in fewer iterations
- Example: Small state spaces where sweeps are cheap

### Practical Reality

In modern RL:
- **Model-free is standard** (we rarely have perfect models)
- **Temporal Difference** methods (next notebook) get best of both worlds:
  - Model-free like MC
  - Low variance like DP (through bootstrapping)
  - Can handle continuing tasks
- **Deep RL** uses TD methods (DQN, A3C, PPO) for complex tasks

**The big insight**: MC taught us we can learn from experience alone. This opened the door to applying RL to real-world problems without needing perfect models!

</details>

---

## Challenge Exercise (Advanced)

**Implement Every-Visit MC Prediction** and compare it with First-Visit MC.

**Question**: How does Every-Visit MC differ from First-Visit in terms of:
1. Number of samples collected per episode
2. Bias and variance properties
3. Convergence speed

Implement it and run experiments to verify your hypothesis!

<details>
<summary>Click to see solution sketch</summary>

```python
def mc_prediction_every_visit(env, policy, gamma, n_episodes):
    """Every-Visit Monte Carlo - count ALL visits to each state."""
    n_states = env.observation_space.n
    returns_sum = np.zeros(n_states)
    returns_count = np.zeros(n_states)
    V = np.zeros(n_states)
    
    for _ in range(n_episodes):
        episode = generate_episode(env, policy)
        G = 0
        
        # No visited set - count EVERY occurrence
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # Update for every visit (no first-visit check!)
            returns_sum[state] += G
            returns_count[state] += 1
        
        for s in range(n_states):
            if returns_count[s] > 0:
                V[s] = returns_sum[s] / returns_count[s]
    
    return V

# Both should converge to same V^π, but every-visit may be faster!
```

</details>

---

print("Congratulations! You've completed Part 4 of the RL Tutorial!")
print("\nKey takeaways:")
print("- Monte Carlo methods learn from complete episodes")
print("- They don't need a model - completely model-free")
print("- MC Prediction estimates V or Q by averaging returns")
print("- MC Control uses ε-greedy or exploring starts for exploration")
print("- MC has high variance but no bias from bootstrapping")
print("\nNext: 05_temporal_difference.ipynb")